In [59]:
import pandas as pd
import numpy as np
import scipy.stats as ss

X_train = pd.read_csv('data/hr_data/X_train.csv')
X_test = pd.read_csv('data/hr_data/X_test.csv')
y_train = pd.read_csv('data/hr_data/y_train.csv')

# DataFrame 전처리 진행(결측치 등)
X_train = X_train.sort_values(by='enrollee_id', ascending=True).reset_index(drop=True)
X_test = X_test.sort_values(by='enrollee_id', ascending=True).reset_index(drop=True)
y_train = y_train.sort_values(by='enrollee_id', ascending=True).reset_index(drop=True)
train_df = X_train.merge(y_train, on='enrollee_id', how='inner')
train_df = train_df.dropna(how='any')
X_train = train_df.iloc[:, :-1]
y_train = train_df.iloc[:, -1]

# 특성 전처리 진행
#X_train['experience'] = pd.to_numeric(X_train['experience'].replace({'>20':'25', '<1':'0'}))
#X_train['company_size'] = (X_train['company_size']
#.replace({'10000+':'10000-20000', '10/49':'10-49', '<10':'1-10'})
#.str.split('-', expand=True)
#.astype('float64').mean(axis=1)
#)
#X_train['relevent_experience'] = X_train['relevent_experience'].replace({'Has relevent experience':1, 'No relevent experience':0})
#X_train['last_new_job'] = X_train['last_new_job'].replace({'never':0, '>4':5}).astype('int64')

def preprocessing_X(df):
    df['experience'] = pd.to_numeric(df['experience'].replace({'>20':'25', '<1':'0'}))
    df['company_size'] = df['company_size'].replace({'10000+':'10000-20000', '10000':'10000-20000', '10/49':'10-49', '<10':'1-10'}).str.split('-', expand=True).astype('int64').mean(axis=1)
    df['relevent_experience'] = df['relevent_experience'].replace({'Has relevent experience':1, 'No relevent experience':0})
    df['last_new_job'] = df['last_new_job'].replace({'never':0, '>4':5}).astype('int64')
    df = pd.get_dummies(df)
    return df

#X_train = preprocessing_X(X_train)
#X_test = preprocessing_X(X_test)

X_test.loc[:, ['company_size', 'company_type']]

#pd.concat([X_train.nunique(), X_train.dtypes], axis=1)
#X_train.isna().sum(), X_train.shape

,company_size,company_type
0,NaN,Pvt Ltd
1,NaN,Pvt Ltd
2,500-999,Pvt Ltd
3,NaN,NaN
4,5000-9999,Pvt Ltd
...,...,...
2121,50-99,Pvt Ltd
2122,500-999,Pvt Ltd
2123,10000,Public Sector
2124,100-500,Pvt Ltd


In [2]:
X_train

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19153,7386,city_173,0.878,Male,No relevent experience,no_enrollment,Graduate,Humanities,14,NaN,NaN,1,42
19154,31398,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,14,NaN,NaN,4,52
19155,24576,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,4,44
19156,5756,city_65,0.802,Male,Has relevent experience,no_enrollment,High School,NaN,<1,500-999,Pvt Ltd,2,97


In [3]:
X_test

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,32403,city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21
1,9858,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98
2,31806,city_21,0.624,Male,No relevent experience,no_enrollment,High School,NaN,<1,NaN,Pvt Ltd,never,15
3,27385,city_13,0.827,Male,Has relevent experience,no_enrollment,Masters,STEM,11,10/49,Pvt Ltd,1,39
4,27724,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10000,Pvt Ltd,>4,72
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,27163,city_173,0.878,Male,No relevent experience,no_enrollment,Masters,STEM,16,<10,Pvt Ltd,>4,58
2122,24507,city_90,0.698,NaN,No relevent experience,no_enrollment,Graduate,STEM,20,NaN,NaN,never,133
2123,21339,city_21,0.624,Female,No relevent experience,Full time course,High School,NaN,3,NaN,NaN,never,56
2124,1289,city_103,0.920,Male,No relevent experience,no_enrollment,Graduate,Humanities,16,NaN,Public Sector,4,15


In [4]:
y_train

,enrollee_id,target
0,8949,1
1,29725,0
2,11561,0
3,33241,1
4,666,0
...,...,...
19153,7386,1
19154,31398,1
19155,24576,0
19156,5756,0
